In [1]:
#Author     : Fan Li
#Update Date: 2022.09.27
#Version    : 2.0

#Intrusion Detection System Using Machine Learning (Xgboost)

#Original Data (cic ids 2017)

#hardware
#24 CPUs | 96GiB | c7.6xlarge.4

#System
#Ubuntu 20.04 server 64bit

In [2]:
#1. Loading Module
#basic
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import gc    #memory manage

import math
import numpy as np
import pandas as pd

#draw figure
import seaborn as sns
import matplotlib.pyplot as plt

#preprocessing
from sklearn.preprocessing import LabelEncoder
from collections import Counter

#Feature Selection

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

#metrics

from sklearn import model_selection,feature_selection,utils,ensemble,metrics

#time
from time import time
import datetime

In [3]:
#2. Reading Dataset
base_path="../../data/"
# Monday benign
f1_path=base_path+"Monday-WorkingHours.pcap_ISCX.csv"
f1=pd.read_csv(f1_path)
#Tuesday benign
f2_path=base_path+"Tuesday-WorkingHours.pcap_ISCX.csv"
f2=pd.read_csv(f2_path)
#Wed benign
f3_path=base_path+"Wednesday-workingHours.pcap_ISCX.csv"
f3=pd.read_csv(f3_path)
#Thur WebAttack
f4_path1=base_path+"Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"
f41=pd.read_csv(f4_path1)
f4_path2=base_path+"Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"
f42=pd.read_csv(f4_path2)
#Fri Benign DDos PortScan
f5_path1=base_path+"Friday-WorkingHours-Morning.pcap_ISCX.csv"
f51=pd.read_csv(f5_path1)

f5_path2=base_path+"Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"
f52=pd.read_csv(f5_path2)

f5_path3=base_path+"Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
f53=pd.read_csv(f5_path3)

In [4]:
data=pd.concat((f1,f2,f3,f41,f42,f51,f52,f53))
data.reset_index(inplace=True,drop=True) # duplicate index exist, reset index
data.shape

(2830743, 79)

In [5]:
data

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,61374,61,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830739,61378,72,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830740,61375,75,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2830741,61323,48,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
#3. Preprocessing
#3.1 remove useless data
#remove duplicate row
data=data.drop_duplicates()
#remove rows with null value and infinite value
data=data.replace(np.inf,np.nan)
data=data.dropna()

In [7]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
print(Counter(y))

Counter({'BENIGN': 2072476, 'DoS Hulk': 172846, 'DDoS': 128014, 'PortScan': 90694, 'DoS GoldenEye': 10286, 'FTP-Patator': 5931, 'DoS slowloris': 5385, 'DoS Slowhttptest': 5228, 'SSH-Patator': 3219, 'Bot': 1948, 'Web Attack � Brute Force': 1470, 'Web Attack � XSS': 652, 'Infiltration': 36, 'Web Attack � Sql Injection': 21, 'Heartbleed': 11})


In [8]:
#Label Encoding
le=LabelEncoder()
le=le.fit(y)
le.classes_
y=le.transform(y)
print(Counter(y))

Counter({0: 2072476, 4: 172846, 2: 128014, 10: 90694, 3: 10286, 7: 5931, 6: 5385, 5: 5228, 11: 3219, 1: 1948, 12: 1470, 14: 652, 9: 36, 13: 21, 8: 11})


In [9]:
#Ram Clean
del data
gc.collect()

0

In [10]:
#3.2 Splitting data
X_train,X_test,y_train,y_test=model_selection.train_test_split(X,y,test_size=0.33,random_state=123456)

In [11]:
print(Counter(y_train))

Counter({0: 1388615, 4: 115556, 2: 85778, 10: 60860, 3: 6923, 7: 3977, 6: 3704, 5: 3506, 11: 2123, 1: 1293, 12: 987, 14: 436, 9: 20, 13: 17, 8: 10})


In [12]:
#4. Training
clf=XGBClassifier(n_estimators=100,random_state=123456,n_jobs=-1)

In [13]:
time1=time()
clf.fit(X_train,y_train)
print("Time of Training : {:.3f} s".format(time()-time1))

Time of Training : 635.581 s


In [14]:
y_pred_train=clf.predict(X_train)
y_pred_test=clf.predict(X_test)

In [15]:
#5. Result 
#5.1 Original Data
print("==Train==")
result=metrics.confusion_matrix(y_train,y_pred_train)
report=metrics.classification_report(y_train,y_pred_train,zero_division=1)

print(result)
print(report)

print("Accuracy Score = ",metrics.balanced_accuracy_score(y_train,y_pred_train))
print("Precison       = ",metrics.precision_score(y_train,y_pred_train,average='macro'))
print("Recall         = ",metrics.recall_score(y_train,y_pred_train,average='macro'))
print("F1_score       = ",metrics.f1_score(y_train,y_pred_train,average='macro'))

==Train==
[[1387855       4       0       0     130       1       0       0       0
        0     624       0       1       0       0]
 [     35    1258       0       0       0       0       0       0       0
        0       0       0       0       0       0]
 [      1       0   85777       0       0       0       0       0       0
        0       0       0       0       0       0]
 [      0       0       0    6919       4       0       0       0       0
        0       0       0       0       0       0]
 [      6       0       0       4  115545       0       0       0       0
        0       0       0       1       0       0]
 [      0       0       0       0       0    3499       7       0       0
        0       0       0       0       0       0]
 [      1       0       0       0       0       1    3702       0       0
        0       0       0       0       0       0]
 [      0       0       0       0       0       0       0    3977       0
        0       0       0       0       0

In [16]:
print("==Test==")
result=metrics.confusion_matrix(y_test,y_pred_test)
report=metrics.classification_report(y_test,y_pred_test,zero_division=1)

print(result)
print(report)

print("Accuracy Score = ",metrics.balanced_accuracy_score(y_test,y_pred_test))
print("Precison       = ",metrics.precision_score(y_test,y_pred_test,average='macro'))
print("Recall         = ",metrics.recall_score(y_test,y_pred_test,average='macro'))
print("F1_score       = ",metrics.f1_score(y_test,y_pred_test,average='macro'))

==Test==
[[683318     86      6      1     78     18      1      0      0      0
     350      1      0      0      2]
 [   123    532      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     6      0  42230      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     4      0      0   3347      9      3      0      0      0      0
       0      0      0      0      0]
 [    19      0      0      5  57259      0      2      0      0      0
       5      0      0      0      0]
 [     6      0      0      1      0   1712      3      0      0      0
       0      0      0      0      0]
 [     5      0      0      1      0      0   1675      0      0      0
       0      0      0      0      0]
 [     2      0      0      0      0      0      0   1951      0      0
       0      0      1      0      0]
 [     0      0      0      0      0      0      0      0      1      0
       0      0      0      0      0]
 

### Xgboost With Original Data
Accuracy Score =  0.9068209863566048
Precison       =  0.9384528923374698
Recall         =  0.9068209863566048
F1_score       =  0.9197695616013088